<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas/pandas_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

176. Second Highest Salary

https://leetcode.com/problems/second-highest-salary/description/

```sql
SELECT max(salary) as SecondHighestSalary
FROM (
SELECT
    id, salary,
    dense_rank() OVER (ORDER BY salary DESC) AS rn
FROM Employee e)
WHERE rn = 2
```

In [4]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})

In [31]:
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    result = employee.assign(rn = employee.salary.rank(method='dense', ascending=False)).query("rn == 2")[['salary']].rename({"salary":"SecondHighestSalary"}, axis=1)
    return result.head(1) if result.shape[0] else pd.DataFrame([{"SecondHighestSalary": None}])